In [2]:
from initial_dataset import InitialDataset
from verstack import LGBMTuner
import h5py
import pandas as pd
import numpy as np
from dataset import ParticleDataset

2023-05-10 20:40:28.553057: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-10 20:40:29.379749: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-10 20:40:29.379850: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-05-10 20:40:29.379858: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [3]:
# Maybe old code.
# def _load_data(name):
#     file_extension_length = len(".h5")
#     with h5py.File(f'{name}', 'r') as f:
#         filename = name.split('/')[-1][:-file_extension_length]
#         return pd.DataFrame(f[filename][:], dtype=np.float32)
#
# path="../data/initial/train.h5"
# variables_path="../data/initial/variables.txt"
# label_name='Truth'
#
# all_data = _load_data(path)
# with open(variables_path, 'r') as file:
#     variables = file.read()
# variables = variables.replace("'", "")
# variables = variables.replace(" ", "")
# variables = variables.replace("\n", "")
# variables = variables.split(",")
# features = all_data[variables]
# labels = all_data[label_name]
#
# print(features.head())
# print(labels.head())

In [4]:
data = ParticleDataset(variables_path="/home/amh/Documents/coding/GitHub/AppliedML2023/data/initial/variables.txt")
train, val = data.split_data(0.8, seed=42)
features, labels = train[:]
features = features.numpy()
labels = labels.numpy()
features = pd.DataFrame(features, columns=data.variables)
labels = pd.Series(labels)

print(features.head())

FileNotFoundError: [Errno 2] No such file or directory: '/home/amh/Documents/coding/GitHub/AppliedML2023/data/initial/variables.txt'

In [ ]:
tuner = LGBMTuner(metric="accuracy",
                  device_type="cpu")
tuner.fit(features, labels)

In [ ]:
feature_importance_list = [(tuner.feature_importances[i], features.columns[i]) for i in range(len(features.columns))]
top_features = [tup[1] for tup in sorted(feature_importance_list)[::-1]]
print(top_features[:15])

In [ ]:
features_reduced = features[top_features]
tuner_reduced = LGBMTuner(metric="accuracy")
tuner_reduced.fit(features_reduced, labels)

In [ ]:
import pickle

with open("/home/amh/Documents/Coding/GitHub/AppliedML2023/data/initial/lgbm_tuner.pickle", "wb") as handle:
    pickle.dump(tuner_reduced, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open("/home/amh/Documents/Coding/GitHub/AppliedML2023/data/initial/lgbm_tuner_full.pickle", "wb") as handle:
    pickle.dump(tuner, handle, protocol=pickle.HIGHEST_PROTOCOL)
print("done")